In [113]:
import nltk
from nltk.cluster.util import cosine_distance
import numpy as np
import difflib

In [33]:
speeches = []
for title in ['MLK','JFK','FDR1','FDR2','RR']:
    filename =  "Speeches/" + title + '.txt'
    file = open(filename).read()
    tokens = nltk.sent_tokenize(file)
    for s in range(len(tokens)):
        tokens[s] = nltk.word_tokenize(tokens[s])
    speeches.append(tokens)

In [120]:
lengths = []

for speech in speeches:
    for sent in speech:
        lengths.append(len(sent))

lengths = np.array(lengths)

lower = np.percentile(lengths,35)
upper = np.percentile(lengths,70)
print(lower,upper)

15.0 27.0


In [155]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [156]:
speeches_pos = []
speeches_syl = []
for speech in speeches: 
    speech_pos = []
    speech_syl = []
    for sent in speech:
        sent_pos = nltk.pos_tag(sent)
        sent_pos = [i[1] for i in sent_pos]
        speech_pos.append(sent_pos)
        sent_syl = []
        for word in sent:
            sent_syl.append(syllable_count(word))
        speech_syl.append(sent_syl)
    speeches_pos.append(speech_pos)
    speeches_syl.append(speech_syl)

In [70]:
def similarity(sent1, sent2):
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

<h1>See similarity between speeches<\h1>

In [118]:
speakers = ['MLK','JFK','FDR1','FDR2','RR']
for j in range(4):
    for k in range(j+1,5):
        s1 = speeches_pos[j]
        s2 = speeches_pos[k]
        sims = []
        diffs = []
        for i in range(20):
            sims.append(similarity(s1[i],s2[i]))
            diffs.append(difflib.SequenceMatcher(None,s1[i],s2[i]).ratio())
        print(speakers[j],speakers[k])
        print(np.mean(sims),np.percentile(sims,25),np.percentile(sims,75))
        print(np.mean(diffs),np.percentile(diffs,25),np.percentile(diffs,75))


MLK JFK
0.6220304846867513 0.4923005233198352 0.7339773058637489
0.3006662279964215 0.20184171229467396 0.37648809523809523
MLK FDR1
0.6146667205967447 0.5410492707088581 0.6881994069902073
0.32166032709134856 0.2406191369606004 0.3946428571428572
MLK FDR2
0.6450252338399329 0.5098634398112862 0.7893458546863694
0.33822507518913714 0.27313211281543787 0.41483516483516486
MLK RR
0.5734285519920796 0.4331064711033647 0.7241515524761672
0.3409956575121285 0.26188455008488964 0.39097744360902253
JFK FDR1
0.6673191519134841 0.603501755903317 0.7785276161796155
0.316748177448371 0.2543604651162791 0.36923076923076925
JFK FDR2
0.576865842494059 0.47262234670390935 0.7555928390503563
0.26630284013765604 0.19345238095238096 0.3106060606060606
JFK RR
0.5705532812408614 0.4922003191214408 0.7089744831949277
0.27695813837984745 0.21465677179962894 0.33695652173913043
FDR1 FDR2
0.6356488471492326 0.5942045827688409 0.705835895986157
0.3269996522838861 0.29962686567164176 0.3857104485629868
FDR1 RR


<h1><font color="yellow"> NOW INPUT TEXT AND SEE HOW SPEECHLIKE IT IS (POS Similarity) <\font><\h1>

In [188]:
# take the input speech
file = open('nonSpeeches/GlobalStudiesCorpus.txt').read()
# file = open('Speeches/911.txt').read()
inputSpeech = nltk.sent_tokenize(file)
for s in range(len(inputSpeech)):
    inputSpeech[s] = nltk.word_tokenize(inputSpeech[s])

pos_inputSpeech = [] 
syl_inputSpeech = []

for sent in inputSpeech:
    sent_pos = nltk.pos_tag(sent)
    sent_pos = [i[1] for i in sent_pos]
    pos_inputSpeech.append(sent_pos)
    sent_syl = []
    for word in sent:
        sent_syl.append(syllable_count(word))
    syl_inputSpeech.append(sent_syl)
        
# loop through each sentence
lexCompare = []
posCompare = []
sylCompare = []

for sent in range(len(inputSpeech)):
    
    if len(inputSpeech[sent]) <= lower:
        percentile = 0
    elif len(inputSpeech[sent]) < upper:
        percentile = 1
    else:
        percentile = 2

    allLex = []
    allPos = []
    allSyl = []    
    
    for speech in range(len(speeches)):
        for sent2 in range(len(speeches[speech])):
            if percentile == 0:
                if len(speeches_pos[speech][sent2]) <= lower:
                    allLex.append(similarity(inputSpeech[sent],speeches[speech][sent2]))
                    allPos.append(similarity(pos_inputSpeech[sent],speeches_pos[speech][sent2]))
#                     allPos.append(difflib.SequenceMatcher(None,pos_inputSpeech[sent],speeches_pos[speech][sent2]).ratio())
                    allSyl.append(difflib.SequenceMatcher(None,syl_inputSpeech[sent],speeches_syl[speech][sent2]).ratio())
            elif percentile == 1:
                if len(speeches_pos[speech][sent2]) > lower and len(speeches_pos[speech][sent2]) < upper:
                    allLex.append(similarity(inputSpeech[sent],speeches[speech][sent2]))
                    allPos.append(similarity(pos_inputSpeech[sent],speeches_pos[speech][sent2]))
#                     allPos.append(difflib.SequenceMatcher(None,pos_inputSpeech[sent],speeches_pos[speech][sent2]).ratio())
                    allSyl.append(difflib.SequenceMatcher(None,syl_inputSpeech[sent],speeches_syl[speech][sent2]).ratio())
            else:
                if len(speeches_pos[speech][sent2]) >= upper:
                    allLex.append(similarity(inputSpeech[sent],speeches[speech][sent2]))
                    allPos.append(similarity(pos_inputSpeech[sent],speeches_pos[speech][sent2]))
#                     allPos.append(difflib.SequenceMatcher(None,pos_inputSpeech[sent],speeches_pos[speech][sent2]).ratio())
                    allSyl.append(difflib.SequenceMatcher(None,syl_inputSpeech[sent],speeches_syl[speech][sent2]).ratio())
    
    lexAvg = np.mean(allLex)
    lexCompare.append(lexAvg)
    
    posAvg = np.mean(allPos)
    posCompare.append(posAvg)
    
    sylAvg = np.mean(allSyl)
    sylCompare.append(sylAvg)
    
print('Lexicon Score')
print(np.mean(lexCompare),np.percentile(lexCompare,25),np.percentile(lexCompare,75))
print('POS score')
print(np.mean(posCompare),np.percentile(posCompare,25),np.percentile(posCompare,75))
print('Syllable scores')
print(np.mean(sylCompare),np.percentile(sylCompare,25),np.percentile(sylCompare,75))

dataForML = [np.mean(lexCompare),np.percentile(lexCompare,25),np.percentile(lexCompare,75),
            np.mean(posCompare),np.percentile(posCompare,25),np.percentile(posCompare,75),
            np.mean(sylCompare),np.percentile(sylCompare,25),np.percentile(sylCompare,75)]
print(dataForML)

Lexicon Score
0.23845892696050544 0.19309686496050005 0.2806807813100855
POS score
0.6656616664554945 0.6323904763031278 0.7155120201934456
Syllable scores
0.5330082241333999 0.5008489488674571 0.5710753141448655
[0.23845892696050544, 0.19309686496050005, 0.2806807813100855, 0.6656616664554945, 0.6323904763031278, 0.7155120201934456, 0.5330082241333999, 0.5008489488674571, 0.5710753141448655]


Next Steps
Analyze more than just pos and pos sequences <<-- in the future we will use both
What else to add:
2.words themselves similarity
1.syllables and syllable sequences
3.size

LBJ
Lexicon Score
0.2206770320260939 0.16444150300872848 0.26751448186390947
POS score
0.58712533150487 0.5050302214253439 0.6801564703521834
Syllable scores
0.5890299449491577 0.5561725977374865 0.6233668124636911


911
Lexicon Score
0.21909331149281452 0.16822689873957603 0.2826479644797504
POS score
0.5679722102197743 0.4622933870068829 0.6686992907910609
Syllable scores
0.5533959611271846 0.5206761695675282 0.5981738554805685


Steve Jobs
Lexicon Score
0.18889641447264988 0.1526232414666126 0.21799046271700828
POS score
0.512222183457417 0.4259241229403694 0.6103960559618823
Syllable scores
0.5890760120927805 0.5519041936890691 0.6362407520634856

MLK (not compared to itself)
Lexicon Score
0.23360487633843435 0.17911676269336066 0.2808387317663956
POS score
0.6114183229340353 0.545222719091455 0.6988506486718966
Syllable scores
0.5780575744862335 0.5469768361534566 0.6084192133145911




take the three metrics as inputs and then use them as inputs for a ml classifier
make a dash app that takes in the text on one side

<h1><font color = "red"> APP STUFF </font></h1>

In [211]:
from sklearn.ensemble import RandomForestClassifier
#Training Data
#order
#LBJ
#911
#BSA
#globalStudies
#Medium
#SethGodin

X = [
    [0.2206770320260939, 0.16444150300872848, 0.26751448186390947, 0.58712533150487, 0.5050302214253439, 0.6801564703521834, 0.5890299449491577, 0.5561725977374865, 0.6233668124636911],
    [0.21909331149281452, 0.16822689873957603, 0.2826479644797504, 0.5679722102197743, 0.4622933870068829, 0.6686992907910609, 0.5533959611271846, 0.5206761695675282, 0.5981738554805685],
    [0.18639193230159604, 0.13879495860288443, 0.20927516051956438, 0.5782069232444341, 0.5024750410242014, 0.6583653027793185, 0.5858910942585438, 0.5664235386626175, 0.6131445223876603],
    [0.22479072982187454, 0.18636712941809297, 0.26951768909072904, 0.6577229024142464, 0.5874365398901084, 0.7356621954845705, 0.5518841529444279, 0.5249945243930422, 0.5796050147073137],
#     [0.17064183606190358, 0.13184682346106283, 0.20599061296683502, 0.49838191720146113, 0.3965624149541385, 0.5987464416853956, 0.5680601664983725, 0.5259715344172728, 0.627135788291258],
    [0.1848718815401489, 0.13693315333732198, 0.2102304448015972, 0.5141976142890523, 0.4084460467781959, 0.6410679832199497, 0.6078637832180149, 0.5805887283151228, 0.6523637058160225]
]

y = [1.0,1.0,0.0,0.0,0.0]
clf = RandomForestClassifier(n_estimators=500, max_depth=None, random_state=720)
clf.fit(X, y)
clf.score(X,y)
clf.predict([[0.17064183606190358, 0.13184682346106283, 0.20599061296683502, 0.49838191720146113, 0.3965624149541385, 0.5987464416853956, 0.5680601664983725, 0.5259715344172728, 0.627135788291258]])

array([0.])